In [ ]:

def rolling_correlation_2ticker(
    ticker1,
    ticker2,
    start_date="1990-01-01",
    end_date=None,
    window_days=252,
    use_adjusted=True,
):
    """
    Calculate rolling correlation between two stocks over a date range.
    
    Parameters:
    - ticker1, ticker2: Stock tickers (e.g., "VTI", "QQQM")
    - start_date, end_date: Date range for analysis
    - window_days: Rolling window size in trading days (default 252 = 1 year)
    - use_adjusted: Use adjusted close (True) or close (False)
    
    Returns:
    - Dictionary with rolling correlation stats and time series
    """
    # 1. Fetch price data for both tickers
    raw_data = yf.download(
        [ticker1, ticker2],
        start=start_date,
        end=end_date,
        progress=False,
        auto_adjust=True,
    )
    
    price_field = "Close"
    if use_adjusted and isinstance(raw_data.columns, pd.MultiIndex) and "Adj Close" in raw_data.columns.get_level_values(0):
        price_field = "Adj Close"
    
    data = raw_data[price_field]
    
    # 2. Get price series and calculate returns
    series1 = data[ticker1].dropna()
    series2 = data[ticker2].dropna()
    
    # Align dates (keep only overlapping period)
    common_idx = series1.index.intersection(series2.index)
    series1 = series1.loc[common_idx]
    series2 = series2.loc[common_idx]
    
    if len(series1) < window_days:
        return {
            "Error": f"Not enough data; need {window_days} days but only have {len(series1)}"
        }
    
    # 3. Calculate returns (log returns more stable)
    returns1 = np.log(series1 / series1.shift(1)).dropna()
    returns2 = np.log(series2 / series2.shift(1)).dropna()
    
    # Align returns
    common_idx = returns1.index.intersection(returns2.index)
    returns1 = returns1.loc[common_idx]
    returns2 = returns2.loc[common_idx]
    
    # 4. Calculate rolling correlation
    rolling_corr = returns1.rolling(window=window_days).corr(returns2)
    
    # 5. Compute summary statistics
    stats = {
        "Ticker 1": ticker1,
        "Ticker 2": ticker2,
        "Start Date": str(rolling_corr.index[0].date()),
        "End Date": str(rolling_corr.index[-1].date()),
        "Window (days)": window_days,
        "Mean Correlation": round(rolling_corr.mean(), 3),
        "Median Correlation": round(rolling_corr.median(), 3),
        "Std Dev Correlation": round(rolling_corr.std(), 3),
        "Min Correlation": round(rolling_corr.min(), 3),
        "Max Correlation": round(rolling_corr.max(), 3),
        "Current Correlation": round(rolling_corr.iloc[-1], 3) if not pd.isna(rolling_corr.iloc[-1]) else np.nan,
    }
    
    return {
        "stats": stats,
        "rolling_correlation": rolling_corr,
    }


In [ ]:
# Rolling correlation between two stocks
result = rolling_correlation_2ticker("SPGP", "VTI", start_date="2000-01-01", end_date=None, window_days=252)
print("Stats:")
for key, value in result["stats"].items():
    print(f"  {key}: {value}")

# Plot rolling correlation over time
rolling_corr_series = result["rolling_correlation"]
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.plot(rolling_corr_series.index, rolling_corr_series.values, linewidth=2)
plt.axhline(y=result["stats"]["Mean Correlation"], color='r', linestyle='--', label=f"Mean: {result['stats']['Mean Correlation']}")
plt.xlabel("Date")
plt.ylabel("Rolling Correlation (1-year window)")
plt.title(f"{result['stats']['Ticker 1']} vs {result['stats']['Ticker 2']} - Rolling Correlation")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
